In [17]:
import modulos as md
import numpy as np
import os
import sys
import constantes as cons
import warnings

def warn(*args, **kwargs):
    pass
warnings.warn = warn
#lista = ['USDCHF=X','GBPUSD=X','EURUSD=X','USDCAD=X','AUDUSD=X','EURGBP=X','EURAUD=X','EURCHF=X','EURJPY=X','GBPCHF=X','CADJPY=X','GBPJPY=X','AUDNZD=X','AUDCAD=X','AUDCHF=X','AUDJPY=X','EURCAD=X','CADCHF=X','NZDJPY=X','NZDUSD=X','CADUSD=X','USDJPY=X','^GSPC','GC=F']
np.seterr(divide='ignore')
lista_filtrada = []
#lista=md.filtradodemonedas ()
lista = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'LINKUSDT', 'ADAUSDT', 'BNBUSDT', 'ATOMUSDT', 'DOGEUSDT', 'SOLUSDT', 'AVAXUSDT', 'MATICUSDT', 'OPUSDT', 'INJUSDT', 'ARBUSDT', 'TIAUSDT', 'ORDIUSDT', 'JTOUSDT']
lista = ['INJUSDT']

timeframe = '1h'
# 0 smart 
# 1 alex
modo = 0
if modo ==0:
    print("####################################### SMART #######################################")
if modo == 1:
    print("####################################### ALEX #######################################")
if len(lista) > 1:
    imprimo = False
    debug = False
else:
    imprimo = True
    debug = True
print(lista)
porcentajes_sumados = 0
win_rate_buenos = 0
win_rate_malos = 0
trades = 0
balance = md.balancetotal()
for symbol in lista:    
    try:
        if modo == 0:
            data = md.estrategia_smart(symbol, debug = False, refinado = False, fuente = 0, timeframe = timeframe, balance = balance, largo = 1)
        if modo == 1:
            data = md.estrategia_alex(symbol, debug = False, refinado = False, fuente = 0, timeframe = timeframe, balance = balance, largo = 6)
        resultado = md.backtesting_smart(data, plot_flag=imprimo, symbol=symbol)
        if resultado['Win Rate [%]'] >= 50:
            lista_filtrada.append(symbol)
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {resultado['Profit Factor']} - Win Rate [%]: {resultado['Win Rate [%]']}")
        porcentajes_sumados=porcentajes_sumados+resultado['Return [%]']
        trades = trades + resultado['# Trades']
        if not np.isnan(resultado['Win Rate [%]']):
            if resultado['Win Rate [%]'] >= 50:
                win_rate_buenos = win_rate_buenos+1
            else:
                win_rate_malos = win_rate_malos+1
        if modo == 0:
            ## para smart
            if ((data.Close.iloc[-1] < (data.decisional_alcista_high.iloc[-2] + (data.atr.iloc[-1]*3)))
                or 
                (data.Close.iloc[-1] > (data.decisional_bajista_low.iloc[-2] - (data.atr.iloc[-1]*3)))
                ):
                print(f"posible entrada symbol: {symbol}")
        if modo == 1:
            ## para alex
            if (    data.sentido.iloc[-1] != -2
                or  data.sentido.iloc[-2] != -2  
                or  data.sentido.iloc[-3] != -2
                or  data.sentido.iloc[-4] != -2
                or  data.sentido.iloc[-5] != -2
                ):
                print(f"posible entrada symbol: {symbol}")
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+"\n")
        pass
for symbol in lista_filtrada:
    md.printandlog(cons.nombrelog,symbol,pal=1)
print(f"\nTimeframe {timeframe} - porcentajes_sumados {md.truncate(porcentajes_sumados,2)} - trades {trades} - win_rate_buenos {win_rate_buenos} - win_rate_malos {win_rate_malos} - Ganancia por trade: {md.truncate((porcentajes_sumados/trades if trades !=0 else porcentajes_sumados),2)}%")


####################################### SMART #######################################
['INJUSDT']
INJUSDT - Return [%]: 13.28% - # Trades: 6 - Profit Factor: 4.54653628767119 - Win Rate [%]: 33.33333333333333

Timeframe 1h - porcentajes_sumados 13.28 - trades 6 - win_rate_buenos 0 - win_rate_malos 1 - Ganancia por trade: 2.21%
